In [2]:
import os
import csv
import pandas as pd

sample_filename = 'sample1000.txt'

dirpath = os.getcwd()
data_foldername = dirpath+ "\data"
filename_path = data_foldername+"\\"+sample_filename


df_data = pd.read_csv(filename_path, header=0)
df_data = pd.DataFrame(df_data)
print(df_data.tail(10))


    VAL_IDEXTCLI
990            .
991            .
992            .
993            x
994            W
995            W
996            w
997            W
998            W
999            w


In [3]:
df_data['siret']=df_data['VAL_IDEXTCLI']


In [4]:
import re
format_siren = r"^[0-9]{9}$"
exp_mdp = re.compile(format_siren)

df_data['siret_9chiffres']= df_data['siret'].apply(lambda x:(exp_mdp.search(x) is not None))

print(df_data.tail(10))

    VAL_IDEXTCLI siret  siret_9chiffres
990            .     .            False
991            .     .            False
992            .     .            False
993            x     x            False
994            W     W            False
995            W     W            False
996            w     w            False
997            W     W            False
998            W     W            False
999            w     w            False


In [6]:
df_9chiffres = df_data[(df_data['siret_9chiffres'] == True)]
print(df_9chiffres)

    VAL_IDEXTCLI      siret  siret_9chiffres
42     444444447  444444447             True
232    539161711  539161711             True
256    794363655  794363655             True
289    793213687  793213687             True
303    797768041  797768041             True
325    791876592  791876592             True
346    290290067  290290067             True
375    478785611  478785611             True
416    350803938  350803938             True
417    000000000  000000000             True
444    533798856  533798856             True
450    503387185  503387185             True
481    808214548  808214548             True
517    798125753  798125753             True
521    531739977  531739977             True
523    000000000  000000000             True
531    753072099  753072099             True
543    514360965  514360965             True
545    820998649  820998649             True
557    435348032  435348032             True
561    501285076  501285076             True
566    410

In [7]:
n = df_9chiffres.count()
print(df_9chiffres.count())

df_9chiffres['siret_9chiffres']= df_9chiffres['siret'].apply(lambda x:(int(x)>0))

print(df_9chiffres[(df_9chiffres['siret_9chiffres'] == True)].count())

VAL_IDEXTCLI       56
siret              56
siret_9chiffres    56
dtype: int64
VAL_IDEXTCLI       53
siret              53
siret_9chiffres    53
dtype: int64


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [23]:
#convert string to array of number
def cleanChaine(chaine9chiffre):
    return str(chaine9chiffre).replace(' ','')

def checkFormat(chaine9chiffre):
    format_siren = r"^[0-9]{9}$"
    exp_mdp = re.compile(format_siren)
    #verification 9 chiffres
    return not (exp_mdp.search(chaine9chiffre) is None)

def checkNotSameNumbers(chaine9chiffre):
    chiffres = {}
    for c in chaine9chiffre:
      if c in chiffres:
        chiffres[c] += 1
      else:
        chiffres[c] = 1
    #print(chiffres)
    #print(len(chiffres.keys()))
    return(len(chiffres.keys())>1)

def isSIRET(chaine9chiffre):
    chaine9chiffre = cleanChaine(chaine9chiffre)

    #verification 9 chiffres
    if (checkFormat(chaine9chiffre)):
        array_chaine9chiffre = [int(i) for i in str(chaine9chiffre)]
        pos = 9
        i=0
        mod = 0
        while i<9 :
            if (pos%2 != 0):
                fac=1
            else:
                fac=2
            #print(array_chaine9chiffre)
            result = int(array_chaine9chiffre[i]) * fac
            #print('pos, fac, result',pos, fac, result)
            
            if (result > 9):
                ar = [int(i) for i in str(result)]
                result=0
                for c in ar:
                    result += int(c)
            #print('result',result)
            mod+=result
            pos-=1
            i+=1
        return (mod%10 == 0)
    else:
        return False

print("checkNotSameNumbers('000000000')",checkNotSameNumbers('000000000'))
print("checkNotSameNumbers('732829320')",checkNotSameNumbers('732829320'))
print("checkNotSameNumbers('552100554')",checkNotSameNumbers('552100554'))

print("isSIRET('000000000')",isSIRET('000000000'))
print("isSIRET('732829320')",isSIRET('732829320'))
print("isSIRET('552100554')",isSIRET('552100554'))


{'0': 9}
1
checkNotSameNumbers('000000000') False
{'7': 1, '3': 2, '2': 3, '8': 1, '9': 1, '0': 1}
6
checkNotSameNumbers('732829320') True
{'5': 4, '2': 1, '1': 1, '0': 2, '4': 1}
5
checkNotSameNumbers('552100554') True
isSIRET('000000000') True
isSIRET('732829320') True
isSIRET('552100554') True


In [24]:
df_9chiffres['siret_valide']= df_9chiffres['siret'].apply(lambda x:(isSIRET(x)))

print(df_9chiffres[(df_9chiffres['siret_valide'] == True)].count())

VAL_IDEXTCLI       51
siret              51
siret_9chiffres    51
siret_valide       51
dtype: int64


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
